# Project Title: Predicting Flight Delays through Machine Learning Classifiers at Scale

Our objective is to help airlines better allocate resources and make business decisions through improved flight delay predictions.

# Group20 - Phase 1 Deliverable

## Section 0: Team Members Details, Phase Leader Plan, Credit Assignment, Citations 
Name: Nathan Chiu \
Email: nchiu20@berkeley.edu \
<img src="https://drive.google.com/uc?export=view&id=1IMdR5dv_Exw3-0wRa3P2Xmu2aZSGUJY5" alt="Google Drive Image" width=15%/>

Name: Dominic Lim \
Email: limdo@berkeley.edu \
<img src="https://drive.google.com/uc?export=view&id=11BDOKqtJgZB0GWXmvJ0k36_MNZh_hK9f" alt="Google Drive Image" width=20%/>

Name: Raul Merino \
Email: raulmy@berkeley.edu \
<img src="https://drive.google.com/uc?export=view&id=1wGRizwNuDBhSet8upm7CauPVZkHdQLRu" alt="Google Drive Image" width=15%/>

Name: Javier Rondon \
Email: javier-rondon@berkeley.edu \
<img src="https://drive.google.com/uc?export=view&id=1cEWCziVbqewNSyjtg361vLdnlZuOn3rf" alt="Google Drive Image" width=15%/>

Team Picture: \
<img src="https://drive.google.com/uc?export=view&id=1I6uOqSQs68yOIiw0gTVPE3XQE4JT_sIK" alt="Google Drive Image" width=15%/>

####Phase Leader Plan

| Phase | Leader | Description |
| --- | --- | --- |
| 1 | Dominic Lim | Project Plan, describe datasets, joins, tasks, and metrics |
| 2 | Nathan Chiu | EDA, baseline pipeline on all data, Scalability, Efficiency, Distributed/parallel Training, and Scoring Pipeline |
| 3 | Raul Merino | Feature engineering + hyperparameter tuning, + in-class review |
| 4 | Javier Rondon | Advanced model architectures and loss functions, select an optimal algorithm, fine-tune & Final report write up  |

#### Credit Assignment

| Task                           | Owner       | Hours (estimate) |
|--------------------------------|-------------|------------------|
| Databricks Cluster and Azure Blob Setup   | Dominic, Javier   |5|
| EDA - Weather                   | Javier, Raul |5|
| EDA - Flights                   | Nathan, Dominic |5|
| Define The Outcome             | Javier |5|       
| Describe Table Join Plan       | Nathan |5|
| Future Steps: ML Pipelines|  Raul |5|

#### Sources used throughout this notebook:
- Peterson, Everett B., et al. “The Economic Cost of Airline Flight Delay.” Journal of Transport Economics and Policy, vol. 47, no. 1, 2013, pp. 107–21. JSTOR, http://www.jstor.org/stable/24396355. Accessed 30 Oct. 2022.
- Aviation Data & Statistics. Aviation Data & Statistics | Federal Aviation Administration. (n.d.). Retrieved October 30, 2022, from https://www.faa.gov/data_research/aviation_data_statistics 
- J. Rondon personal communication, October 27, 2022
- https://www.sciencedirect.com/topics/engineering/wet-bulb-temperature 
- https://education.nationalgeographic.org/resource/barometer
- https://cardinalscholar.bsu.edu/bitstream/handle/123456789/200785/Algarin%20BallesterosJ_2017-3_BODY.pdf?sequence=1&isA llowed=n
- https://engineering.berkeley.edu/news/2010/11/flight-delays-cost-more-than-just-time/
- https://www.airlines.org/dataset/u-s-passenger-carrier-delay-costs/

## Section 1: Abstract

Delays in commercial aviation are frequent and expensive. With roughly 20% of all flights being categorized as being delayed by more than 15 minutes, the downstream costs to the airlines and passengers can total tens of billions of dollars annually. Utilizing Flight Data from the Bureau of Transportation Statistics and Weather Data from the National Oceanic and Atmospheric Administration, our Team is tasked to predict U.S. domestic flight delays two hours before the scheduled departure time.

Our primary use case for flight delay prediction is Airline resource planning and allocation. With only a two-hour notice, airlines would be unable to reposition planes or pilots as contingencies to deal with irregular operations must be put in place 72 hours before departure. However, airlines would be able to allocate corporate resources in anticipation of the fallout from delays (i.e., surge resourcing for help and call centers).

Once we have transformed the data after an initial Exploratory Data Analysis, we plan on joining our datasets utilizing composite keys, with the ultimate goal of using machine learning algorithms such as Decision Trees, Random Forests, and Neural Networks to predict flight delays. We plan on evaluating the performance of our model by comparing the F-2 metric of our machine learning models and a baseline model. With the majority of flights being classified as not delayed, our baseline model will be predict all flights as being `not delayed`.

## Section 2: Data Description (Include Visualizations)

##### Flight Data

The primary data source is the Flight Dataset from the Bureau of Transportation Statistics. The data covers flights from 2015 to 2021 and includes 109 features. The Flight Data provides us with the 2 potential response variables:

- `DEP_DEL15` :  Classification of the flight being delayed by greater than 15 minutes
- `DEP_DELAY`:  Minutes elapsed between Scheduled and Actual Departure Time

The dataset includes both numerical and categorical features that could be useful predictor variables. Numerical features include the time (`CRS_DEP_TIME`), date (`FL_DATE`) and distance flown for the flights. Categorical features such as Airline Carrier (`OP_CARRIER`), Plane Identifier (`TAIL_NUM`) are also of interest.

We conducted an exploratory data analysis of the 3-month sample flight dataset covering the 1st quarter of 2015 (2.8 million recorded flights). The EDA was conducted with a special focus on computing % of missing values per feature, understanding the distribution, scale and range of values of the features.

As a first step, we decided to drop any features with more than 50 percent of missing values from the dataset. The resulting dataset shrank the dimensions of the dataset from 109 features to 56 features. Of note, 49 of the dropped features were primarily related to flights diverted away from scheduled airports (99.74% missing values). We also decided to filter out cancelled flights that did not computed minutes in delay.

Table 1 Missing Values from Flight Data

<img src="https://drive.google.com/uc?export=view&id=1k8ZVVr6Rsmg6NRzcGDtqAssJ2fskefHk" alt="Google Drive Image" width=30%/>

From calculating the proportion of the Delayed and Not-Delayed classes from the predictor variable, `DEP_DEL15`, we observe that the majority of flights are classified as "not delayed". Our sample dataset corroborates our literature review that approximately 20% of all flights are delayed.

Table 2 Proportion of Flights that are Delayed (≥ 15 minutes)

|DEP_DEL15|cnt_per_group|perc_of_count_total|
|---------|-------------|-------------------|
|      0.0|      2166530|  79.58653046470691|
|      1.0|       555702| 20.413469535293096|

We observe from the distribution of `DEP_DELAY` that amongst flights with delays, the majority of delays are less than 30 minutes. It is important to note that negative values are computed for flights that depart early than scheduled. 

Figure 1 Distribution of flight delays in minutes

<img src="https://drive.google.com/uc?export=view&id=1yUw1L3z5efGxp6rv0lLNpIdR56Qk-9_7" alt="Google Drive Image" width=30%/>

Amongst the predictor variables in our dataset, we are particularly interested in the scheduled departure time (`DEP_TIME_BLK`), scheduled departure day of week (`DAY_OF_WEEK`), and Airlines Carrier (`OP_CARRIER`). We parsed the scheduled-departure hour of delayed flights and we observe that as the day progresses, the percentage of flights that are delayed steadily increase. We can speculate that there are network effects whereby earlier delayed flights affect later scheduled flight. This might be a result of delayed scheduled flights taking up Airport Terminal and Gate capacity.

Figure 2 Distribution of flight delays by Scheduled Departure hour

<img src="https://drive.google.com/uc?export=view&id=1D35X80alBoG3aM1N-TUi57aJBP1hSyp0" alt="Google Drive Image" width=50%/>

As it relates to the day-of-the-week of scheduled flights, we observe upticks in flight delays as a % of total flights on Monday, Thursday, Friday, and Sunday. This may be a result of increased  demand and stress on the Airline/Airport systems on days when passengers are more likely to be traveling for business (Monday - Thu/Friday) split and weekend travellers departing and returning from a trip (Friday/Sunday). 

Figure 3 Distribution of flight delays by "Day of Week"

<img src="https://drive.google.com/uc?export=view&id=15-jlbGqaswTMHUNzD4K4Z5lEABZz9w4o" alt="Google Drive Image" width=30%/>

Airline Carrier is a notable categorical variable and is of particular interest. We can see a substantial range in flight timeliness performance by Airline Carrier. The worst performing Airline, Frontier Airlines (`F9`) is followed by Envoy Airlines/American Eagle (`MQ`) and JetBlue (`B6`). An interesting problem that we may need deal with is Airline Carriers that consolidate via mergers/acquisitions or are out of business.

Figure 4 Distribution of flight delays by Airline Carrier

<img src="https://drive.google.com/uc?export=view&id=1Yl5G9j3wyYda_afJ1oZ65deSjDTZ9d4T" alt="Google Drive Image" width=30%/>

d

##### Weather Data

The second data source is the Local Climatological Data (LCD) dataset from the National Centers for Environmental Information. The data covers the period from January 2015–December 2021). The LCD data contains summaries of climatological conditions from weather stations managed by the NWS, FAA, and DOD. Since the weather impacts airline flight operations, it is reasonable to assume that weather features may have predictive power in our model for flight delays.

We conducted an exploratory data analysis (EDA) on the weather set to spot anomalies, calculate the number of missing values, understand the distribution, scale, and range of values of the features, and propose how the features may have explanatory value for our model.

 The data includes hourly observations of temperature, humidity, wind direction and speed, barometric pressure, sky condition, visibility, and weather phenomena. These are critical observations of the atmosphere that help forecasters predict the weather.  

We reviewed the contents of a 3-month sample of the LCD dataset covering January- April 2015. The observations were filtered to correspond to 2799 weather stations located in the US and territories. Overall the dataset contains 124 columns and 14.5 million rows.

We decided to drop any features with more than 50 percent of missing values from the dataset. The resulting dataset contained 20 columns and 14.5 million rows. Table 1 shows the dataset's features and the number of missing values.

Table 1 Missing Values from weather data \
<img src="https://drive.google.com/uc?export=view&id=1Tf_1jOEI_XBA2EtVx8OoJjDC6DvpYVFd" alt="Google Drive Image" width=30%/>

The weather data set contains three features for temperature observations, wet bulb, dry bulb, and dew point temperature. The difference between wet bulb and dry bulb temperatures is a measure of the humidity of the air. The higher the difference in these temperatures, the lower the humidity. The dew point is the temperature under which water vapor condenses. The higher the dew point, the higher the moisture in the air. Figure 1 shows a statistical summary of the three temperature features. The units of temperature are in degrees Fahrenheit.

Figure 1 Statistical summary temperature features \
<img src="https://drive.google.com/uc?export=view&id=19cvzBDReKthCRx5DnBjdP_qq8ZEA-tlC" alt="Google Drive Image" width=70%/>

Figure 2 shows a statistical summary of the pressure features in the data set. The 'HourlyStatonPressure' is the atmospheric pressure observed at the station and reported in inches of Mercury (in Hg). The 'Altimeter setting' is the atmospheric pressure converted to sea level datum and reported in inches of Mercury (in Hg). Rapid drops in atmospheric pressure measurements are associated with cloudy, rainy, or windy weather, while rapid increases are associated with clear skies.

Figure 2 Statistical summary pressure features \
<img src="https://drive.google.com/uc?export=view&id=1MohYdajBcUxgvEp3bysRt4Vf91hhnQoh" alt="Google Drive Image" width=70%/>

Figures 3 and 4 show the statistical summary of the wind-related features in the data set. Strong surface winds, such as crosswinds and gusts, can impact aircraft coming to land or takeoff, causing disruptions to airport operations. Some anomalous high wind speed values were removed, and most observations are less than ten mph winds. The most significant frequency for wind direction was less than 36 degrees. 

Table 2  Statistical summary of wind features \
<img src="https://drive.google.com/uc?export=view&id=1DJnTI3BgsOELhKR6x5X946vz-sADELKf" alt="Google Drive Image" width=50%/>

Figure 3 Distribution of wind speed and direction \
<img src="https://drive.google.com/uc?export=view&id=1HeixtdGdiLrh8a4c3pEzoVdkuIa-vJMo" alt="Google Drive Image" width=70%/>
Figure 4 shows the statistical summary of the visibility feature. For meteorology, this is the horizontal distance an object can be seen and identified in whole miles. The aviation industry uses visibility from the control tower and the pilots' visibility by looking at the runway markings. Weather changes that affect visibility in the control tower or runway will lead to disruption in airport operations, delays, or cancellations. There are some anomalous high values of visibility in the dataset, but most values are around 10 miles.

Table 3 Statistical summary of visibility features \
<img src="https://drive.google.com/uc?export=view&id=1NlhuICOwr1gxsavW0hWtgjnhMlfwv4ug" alt="Google Drive Image" width=40%/>

Figure 5 shows the distribution of entries per station in a 3 month period. If a station reports their data once every hour, they should have about 2160 entries in the dataset, which is clearly not the case for the majority of the stations. A lot of them seem to report their data every 20 minutes, which is why there're a lot of stations in the 6300 - 6750 bucket. There's also a not insignificant number of stations that seem to report their data every 5 minutes. There're also a lot of stations who report with a lower frequency that once an hour on average. This may pose a challenge when deciding how to join the weather data to the flight data, as some stations may not have recent enough data, and it may not be uniform across all flights

Figure 5 Reporting Frequency by Station

<img src="https://drive.google.com/uc?export=view&id=1G_ZWaStDVjX1r6aa2KHMbwfOB0WJBenY" alt="Google Drive Image" width=40%/>

Figure 6 shows the pearson correlation between the previously discussed variables. As expected, all 3 temperature measurements, despite their differences, are highly positively correlated, which likely means that including more than one of them will not provide the model with more information. Humidity and Visibility also have a relatively high correlation, although negative in this case. As humidity increases, visibility decreases, as it becomes harder to see longer distances.

Figure 6 Pearson Correlation Matrix for Hourly Weather Data

<img src="https://drive.google.com/uc?export=view&id=1fV7nV_Wao9CCXSQ8KoP30JCbfq2Oh9gY" alt="Google Drive Image" width=40%/>

## Section 3: Machine Algorithms and Metrics

An important criteria in selecting machine learning algorithms to implement at scale is the algorithm’s parallelizability because it enables us to run our regression and classification on a large dataset with minimal runtimes. 

A common algorithm used to classification problems is K nearest neighbors (KNN). This approach uses similarity based on data points that lie nearby the sample data point. KNN employs lazy evaluation, meaning it doesn’t call the dataset until it is needed. The downside to this is that running KNN becomes computationally expensive, especially for large datasets. This can be resolved by caching, but the size of the flights and weather dataset renders this infeasible. Moreover, since the algorithm involves looking at nearby points and different centroids, for a large K, the algorithm makes many passes over the data. Therefore, KNN is not very parallelizable and will not be used.

The next approach we considered is decision trees, which is a model that starts with a single point and then branches out by decisions and their subsequent outcomes. With large datasets, we can use the MapReduce framework using each node of decisions as the split. The structure of decision trees is conducive to parallelization because we can represent each level of the tree as a MapReduce job, meaning we can represent data in a decision tree very efficiently. 

Similar to decision trees, random forests are collections of decisions trees that impart the benefits of decisions trees with additional benefits. Random forests are an ensemble operation, enabling us to test multiple hypotheses with multiple decision trees efficiently. In operation, we would take the training dataset then split it into different partition. Then each partition would have multiple decision trees mapped to different output files. We would then merge these files into the random forest. Given the efficiency at scale and expected performance, we are leaning towards using random forests. 
We will also consider neural networks (NN) and boosted trees via methods like ADA.

#####Metrics  
We selected the metrics to evaluate the performance of our models considering the business impact on our customer (the airline) due to decisions made based on incorrect predictions from our model. In addition, since we are using a classification model, we considered the ramifications on the business from false positives and false negatives. 

We considered the costs incurred by the airline attributed to delays. Airlines must bear the additional costs from their crew, costs for accommodating disrupted passengers, and the costs of aircraft re-positioning. 
There is also a cost of lost demand from potential customers who consider an airline's reputation for flight delays and customer service before purchasing. Given the competitive nature of the business on specific domestic markets, this cost may be substantial.

While our model cannot be used to mitigate the costs of most of these factors, our model can assist the airline in improving its reputation for customer satisfaction in case of delays that disrupt travel. Taking proactive measures to assist passengers may positively impact reputation and have a measurable favorable effect on sales and customer loyalty.

The consequences of our model errors in predictions can be summarised as follows:

- False positive: The model predicted a delay for the flight, but the flight departed on time. The airline incurred additional expenses to meet a demand that did not materialize. There is no impact on the airline's reputation with its customers. 

- False negative: The model predicted an on-time departure for the flight, but the flight was delayed. The airline did not allocate resources to handle the surge in demand for staff. Passengers are negatively affected by the wait time to obtain information from the airline, and the airline's reputation with its customers suffers. 

The impact of these errors depends on the number of passengers and the customer mix of the flight (share of premium passengers and economy passengers). For example, a mainline carrier operating a route with high competition, such as JFK-LAX, with many first-class and business passengers paying the highest fares, may be more interested in avoiding false negative results to retain the loyalty of premium passengers.
On the other hand, low-cost carriers operating a leisure route, such as JFK-MCO, may be more interested in reducing customer service costs and prefer a model with fewer false positives.

We can use a Fbeta score as a metric, which will allow us to consider giving more weight to either recall or precision to address the needs of our airline.

Here is the formal definition of F Beta:

<img src="https://drive.google.com/uc?export=view&id=1VqqQB1KwiL1oa3fSrygezE_xSP2hzlwC" alt="Google Drive Image" width=40%/>

#####Baseline Models

We will consider two baseline models:
- Logistic regression
- Assume `NO DELAY` classification

#####Project Key Steps

<img src="https://drive.google.com/uc?export=view&id=1e4xOC48fziq9fNtN8--d68PaXARw4LoT" alt="Google Drive Image" width=60%/>


Block Diagram for Modeling Workflow. ( from Kurt Eulau on Slack) \
<img src="https://drive.google.com/uc?export=view&id=1nj8VFDpJvMnU7suoZCZqx5KJ8rsKD_vJ" alt="Google Drive Image" width=60%/>

## Section 4 Machine Learning Pipelines

## Section 4a: Machine Learning Joins

To join the flights and weather data, we need to figure out the closest weather station to a given airport through minimizing distance. However, the airport data does not have the longitude and latitude data that the weather dataset has so we need to pull in longitude and latitude from an external dataset from the global airport dataset [here](https://www.partow.net/miscellaneous/airportdatabase/index.html). Overall, we plan on starting with the airlines dataset and then left joining the global airport code dataset, the station dataset, and finally the weather dataset. Once we have all the datapoints we need in one table, then we can minimize the distance between the airport and weather and the timestamps between the flight and weather observation after standardizing all dates to UTC.

Here are the steps we plan on following with some SQL pseudocode:
  1) Left join the global airport dataset onto trimmed flights dataset with origin airport and timestamp on the three letter codes in both datasets e.g. ATL for Atlanta
  
``` sql
WITH flights_lat_long AS (
SELECT 
  da.origin,
  da.dep_time,
  dga.IATA,
  dga.ICAO,
  dga.longitude,
  dga.latitude
FROM df_airlines da 
LEFT JOIN df_global_airports dga 
  ON da.Origin = dga.IATA
),
```
  
  2) Left join the station table onto the above table to pull in station_id to get the station data then take the min distance to neighbor
  
  ``` sql
flights_station AS (
SELECT 
  fll.*,
  ds.station_id,
  ds.neighbor_call,
  ds.distance_to_neighbor
FROM flights_lat_long fll
LEFT JOIN df_stations ds 
  ON fll.ICAO = ds.neighbor_call --Join on four letter acronym
),

neighbor_min (
SELECT
  fs.station_id,
  MIN(fs.distance_to_neighbor) AS distance_to_neighbor_min
FROM flights_station fs
GROUP BY 1
),

flight_stations AS (
SELECT
  fs.*
FROM flight_station fs
JOIN neighbor_min nm
  ON fs.distance_to_neighbor = nm.distance_to_neighbor_min
)
```
  3) Left join the weather data in on neighbor_call, which is the closest airport to the weather station
  
  ``` sql
SELECT 
  fs.*,
  dw.*,
  DATEDIFF('minutes',fs.dep_time, ds.date) --Look at min date difference to account for time
FROM flights_stations fs
LEFT JOIN df_weather dw 
  fs.station_id = dw.station
```

To summarize, the flow is: airport code | dep time UTC >> airport code | dep time UTC | closest_neighbor >> airport code | dep time UTC | closest_neighbor | weather station data

Note that we have to take the min of neighbor distance and minimize the time difference between the departure time and weather observation
  
We will also make a composite key of the airport code and the departure timestamp to streamline the joining operation.

## Section 4b: Splitting the data

Given that we are dealing with a time series we can't just randomly assign the data to either the training or test set. Every flight from 2021 will be assigned to the blind test set, which will only be used for the final evaluation of the model.

The rest of the flights (2015 to 2020) will be used for the training set. We will conduct cross validation by implementing a blocking split. The 6 year period will be divided into 6 parts, one for each year in the dataset. Each of those parts will be split into two, the first 10 months will be used for training while the remaining 2 will be the dev set. By having the last 2 months of the year in the dev set we will avoid any data leakage.

The training set could be further split into a validation set although similarly to the test set, the validation set will be from the later part of the time series. Any normalisations of the data will be done on the training set, and then those values will be applied to the dev and test set in order to avoid any data leaks.

## Section 5: Next Steps:

We will conduct the following tasks progressively as the project advances:
    
- Feature Engineering
- Hyperparameter Tuning
- Preparation of In-Class Presentation
- Selection of Optimal Algorithm 
- Write Final Report
- Final Presentation

Links to EDA notebooks

Flight EDA

https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/844424046180911/command/4295587629773808

https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4295587629770423/command/4295587629773761

Weather EDA

https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/364123876153045/command/4295587629773430

https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/364123876153144/command/364123876153145

## Section 6: Open Issues and Problems

We have the following open items under active discussion:


- We need decide how we will deal with missing values in the features 
- How will we create the composite key to join the weather and flight data, given the variety of reported frequencies in the weather data.